In [ ]:
from google.colab import files

# Upload files
uploaded = files.upload()

import zipfile
import io

zip_ref = zipfile.ZipFile(io.BytesIO(uploaded['images.zip']), 'r')
zip_ref.extractall('/content') # 압축 해제한 파일 위치
zip_ref.close()

Saving images.zip to images.zip


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 포켓몬 이미지가 저장된 기본 디렉토리
base_dir = '/content/images'

# Image data generator with some data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

# Flow training images in batches
train_generator = train_datagen.flow_from_directory(base_dir, target_size=(150, 150), batch_size=32, class_mode='categorical')

# CNN 모델
model = tf.keras.models.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# 훈련
history = model.fit(train_generator, epochs=5)


Found 547 images belonging to 50 classes.


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/5
18/18 [==============================] - 38s 2s/step - loss: 3.9979 - accuracy: 0.0201
Epoch 2/5
18/18 [==============================] - 33s 2s/step - loss: 3.7701 - accuracy: 0.0347
Epoch 3/5
18/18 [==============================] - 44s 2s/step - loss: 3.5148 - accuracy: 0.0548
Epoch 4/5
18/18 [==============================] - 33s 2s/step - loss: 3.1242 - accuracy: 0.1389
Epoch 5/5
18/18 [==============================] - 33s 2s/step - loss: 2.8284 - accuracy: 0.1974


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 입력 이미지 업로드
uploaded_test_images = files.upload()

# 업로드한 이미지 파일명 가져오기
test_image_name = list(uploaded_test_images.keys())[0]  # 첫 번째 업로드한 이미지 이름

# 이미지 로드 및 사이즈 조정
test_image = load_img(test_image_name, target_size=(150, 150))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# 클래스 예측
result = model.predict(test_image)
predicted_class = np.argmax(result, axis=1)

# 해당 클래스의 레이블 가져오기
label_map = train_generator.class_indices
predicted_label = list(label_map.keys())[list(label_map.values()).index(predicted_class)]


Saving 2.jpg to 2.jpg
1/1 [==============================] - 0s 151ms/step


In [ ]:
import pandas as pd

# 포켓몬 CSV 파일 업로드 및 정보 추출
uploaded_csv = files.upload()
try:
    pokemon_df = pd.read_csv('pokemon.csv')
except UnicodeDecodeError:
    pokemon_df = pd.read_csv('pokemon.csv', encoding='ISO-8859-1')  # 엑셀 파일 깨짐 해결 encoding='ISO-8859-1'

# 예측된 클래스 번호를 기반으로 포켓몬 정보 찾기
predicted_no = predicted_class[0]
predicted_pokemon_info = pokemon_df[pokemon_df['No.'] == predicted_no].iloc[0]

# 예측 확률 계산
predicted_probability = np.max(result) * 100

# Type1 및 Type2 정보 가져오기 (없는 경우 'None'으로 처리)
type1 = predicted_pokemon_info['kor_type_1'] if not pd.isna(predicted_pokemon_info['kor_type_1']) else 'None'
type2 = predicted_pokemon_info['kor_type_2'] if not pd.isna(predicted_pokemon_info['kor_type_2']) else 'None'

# 결과 출력
print(f"{predicted_probability:.2f}% 확률로 {predicted_no}번 포켓몬 \"{predicted_pokemon_info['kor_name']}\"이고, Type1은 \"{type1}\"이고, Type2는 \"{type2}\"입니다.")


Saving pokemon.csv to pokemon (6).csv
100.00% 확률로 46번 포켓몬 "ÆÄ¶ó½º"이고, Type1은 "¹ú·¹"이고, Type2는 "Ç®"입니다.
